# Clean Dataset

In [ ]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install scikit-learn

   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   -- ------------------------------------- 0.5/8.7 MB 4.6 MB/s eta 0:00:02
   -------------------------- ------------- 5.8/8.7 MB 19.9 MB/s eta 0:00:01
   ---------------------------------------- 8.7/8.7 MB 22.6 MB/s  0:00:00
   ---------------------------------------- 0.0/38.5 MB ? eta -:--:--
   ----- ---------------------------------- 5.0/38.5 MB 28.3 MB/s eta 0:00:02
   ----------- ---------------------------- 11.5/38.5 MB 28.0 MB/s eta 0:00:01
   ----------------------- ---------------- 23.1/38.5 MB 37.7 MB/s eta 0:00:01
   ------------------------------------- -- 35.7/38.5 MB 43.3 MB/s eta 0:00:01
   ---------------------------------------  38.3/38.5 MB 44.0 MB/s eta 0:00:01
   ---------------------------------------- 38.5/38.5 MB 34.6 MB/s  0:00:01

   ---------------------------------------- 0/4 [threadpoolctl]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- -----------------------------

In [63]:
import pandas as pd
import os
import re
import unicodedata

In [49]:
# Define threshold (0.5 means drop rows with more than 50% nulls in target columns)
null_threshold = 0.5

# Load your dataset
df = pd.read_csv("Dataset-v2.csv", encoding="latin1")

target_columns = [
    "Tmax", "Wmax", "Lmax", "TempPref_min", "TempPref_max", "TempRange",
    "MeanFecundity", "MinFecundity", "MaxFecundity",
    "WB_pH", "WB_Salinity", "WB_Dissolved Oxygen (mg/L)",
    "WB_Biochemical Oxygen Demand (mg/L)", "WB_Turbidity (NTU)", "WB_Temperature"
]

# Drop rows where more than threshold % of target_columns are missing
row_null_fraction = df[target_columns].isnull().sum(axis=1) / len(target_columns)
rows_to_drop = row_null_fraction[row_null_fraction > 0.5].index
df_cleaned = df.drop(rows_to_drop)

# Drop Unnamed columns and Sources
df_cleaned = df_cleaned.drop(columns=[col for col in df_cleaned.columns if "Unnamed" in col or col == "Sources "], errors="ignore")

print(f"\nDropped {len(rows_to_drop)} rows with >{null_threshold:.0%} missing values in target columns.")

# Save with explicit filename in same folder
output_file = "Dataset_v2_cleaned_50.csv"
df_cleaned.to_csv(output_file, index=False, encoding="utf-8")

print("✅ Cleaned dataset saved as:", output_file)
print("Rows after cleaning:", len(df_cleaned))



Dropped 2538 rows with >50% missing values in target columns.
✅ Cleaned dataset saved as: Dataset_v2_cleaned_50.csv
Rows after cleaning: 2342


In [50]:
df = pd.read_csv("Dataset_v2_cleaned_50.csv", encoding="utf-8")
df.head()

,Fish ID,Species,Common Name,Lake Name,Kingdom,Phylum,Class,Order,Family,Genus,...,MinFecundity,MaxFecundity,Trophic Level Estimate,Trophic Level,WB_pH,WB_Salinity,WB_Dissolved Oxygen (mg/L),WB_Biochemical Oxygen Demand (mg/L),WB_Turbidity (NTU),WB_Temperature
0,F0001,Copella arnoldi,Splash tetra,Amazon Basin,Animalia,Chordata,Actinopterygii,Characiformes,Lebiasinidae,Copella,...,150.0,200.0,NaN,3.26,5.5,0,5.5,2,4,27
1,F0001,Copella arnoldi,Splash tetra,Demerara River Wismar,Animalia,Chordata,Actinopterygii,Characiformes,Lebiasinidae,Copella,...,150.0,200.0,NaN,3.26,6.2,0,6,3,6,27
2,F0001,Copella arnoldi,Splash tetra,Yarakita River,Animalia,Chordata,Actinopterygii,Characiformes,Lebiasinidae,Copella,...,150.0,200.0,NaN,3.26,5.3,0,5,2,3,26.5
3,F0001,Copella arnoldi,Splash tetra,Arauau River,Animalia,Chordata,Actinopterygii,Characiformes,Lebiasinidae,Copella,...,150.0,200.0,NaN,3.26,5.4,0,5.2,2,4,27
4,F0001,Copella arnoldi,Splash tetra,"Kurupung River, Upper Mazaruni District",Animalia,Chordata,Actinopterygii,Characiformes,Lebiasinidae,Copella,...,150.0,200.0,NaN,3.26,6,0,4.5,4,20,26


# Keep min/max of columns with ranges

In [82]:
df = pd.read_csv("Dataset_v2_cleaned_50.csv", encoding="utf-8")

# columns you said contain ranges
range_cols = [
    "WB_pH", "WB_Salinity", "WB_Dissolved Oxygen (mg/L)",
    "WB_Biochemical Oxygen Demand (mg/L)", "WB_Turbidity (NTU)", "WB_Temperature"
]

def normalize_dashes(text):
    if pd.isna(text):
        return text
    # Replace various Unicode dashes with a normal hyphen
    return re.sub(r"[–—-]", "-", str(text))

def normalize_text(s):
    """Normalize unicode dashes/spaces and strip extraneous chars."""
    if pd.isna(s):
        return s
    s = str(s)
    s = unicodedata.normalize("NFKC", s)              # normalize weird unicode
    # replace various dash/minus characters with a normal hyphen
    for ch in ("\u2013", "\u2014", "\u2212", "–", "—"):
        s = s.replace(ch, "-")
    s = s.replace(" to ", "-")     # '10 to 15' -> '10-15'
    s = s.replace("–", "-")
    # remove surrounding punctuation that may be trailing/leading
    s = s.strip().strip(",; ")
    return s

def split_range_cell(s, strategy="first"):
    """
    Parse a cell s and return (min, max) as floats or (None, None).
    strategy:
        - "first": take the first numeric range found (default)
        - "span": take min of all lows and max of all highs (cover span)
        - "mean": take mean of lows and mean of highs
    """
    s2 = normalize_text(s)
    if pd.isna(s2) or s2 == "":
        return (None, None)
    
    # find explicit ranges like "10-15" (with optional decimals and negative)
    range_matches = re.findall(r'(-?\d+(?:\.\d+)?)\s*-\s*(-?\d+(?:\.\d+)?)', s2)
    if range_matches:
        pairs = [(float(a), float(b)) for a, b in range_matches]
        if strategy == "first":
            return pairs[0]
        elif strategy == "span":
            lows = [a for a, b in pairs]
            highs = [b for a, b in pairs]
            return (min(lows), max(highs))
        elif strategy == "mean":
            lows = [a for a, b in pairs]
            highs = [b for a, b in pairs]
            return (sum(lows)/len(lows), sum(highs)/len(highs))
    
    # if no explicit ranges, try to extract numbers in the string
    nums = re.findall(r'(-?\d+(?:\.\d+)?)', s2)
    if nums:
        numsf = [float(x) for x in nums]
        if len(numsf) == 1:
            return (numsf[0], numsf[0])        # single number -> min==max
        else:
            # multiple numbers but no dash: assume pairings left-to-right
            return (numsf[0], numsf[1])
    
    # nothing found
    return (None, None)

# APPLY to dataframe, producing _min and _max for each range column
for col in range_cols:
    if col in df.columns:
        # if column already numeric, just copy to min/max
        if pd.api.types.is_numeric_dtype(df[col]):
            df[col + "_min"] = df[col].astype(float)
            df[col + "_max"] = df[col].astype(float)
        else:
            # choose strategy: "first" is safest when there are multiple ranges in a cell
            df[[col + "_min", col + "_max"]] = df[col].apply(lambda x: pd.Series(split_range_cell(x, strategy="first")))
    else:
        print(f"Warning: column '{col}' not found in df")


for col in range_cols:
    min_col = col + "_min"
    max_col = col + "_max"
    
    if min_col in df.columns and max_col in df.columns:
        # Swap where min > max
        swap_mask = df[min_col] > df[max_col]
        df.loc[swap_mask, [min_col, max_col]] = df.loc[swap_mask, [max_col, min_col]].values


# Replace obvious non-numeric placeholders with NaN
placeholders = ['nr', '-', '\x96', '\x97']
for col in range_cols:
    if col in df.columns:
        df[col] = df[col].replace(placeholders, pd.NA)

# Quick diagnostics: how many missing in each new min/max
print("\nMissing counts after splitting:")
for col in range_cols:
    if col in df.columns:
        print(col, "min NA:", df[col + "_min"].isna().sum(), "max NA:", df[col + "_max"].isna().sum())

# Show sample rows where a particular column failed to parse
example_col = "WB_pH"
if example_col in df.columns:
    mask = df[example_col + "_min"].isna() & df[example_col].notna()
    if mask.any():
        print(f"\nSample problematic '{example_col}' cells (original -> parsed):")
        display(df.loc[mask, [example_col]].head(20))



Missing counts after splitting:
WB_pH min NA: 109 max NA: 109
WB_Salinity min NA: 249 max NA: 249
WB_Dissolved Oxygen (mg/L) min NA: 177 max NA: 177
WB_Biochemical Oxygen Demand (mg/L) min NA: 1092 max NA: 1092
WB_Turbidity (NTU) min NA: 807 max NA: 807
WB_Temperature min NA: 112 max NA: 112


In [83]:
print(df.columns.tolist())
df.head()

['Fish ID', 'Species', 'Common Name', 'Lake Name', 'Kingdom', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Statues', 'FeedingType', 'Tmax', 'Wmax', 'Lmax', 'TempPref_min', 'TempPref_max', 'TempRange', 'MeanFecundity', 'MinFecundity', 'MaxFecundity', 'Trophic Level Estimate', 'Trophic Level', 'WB_pH', 'WB_Salinity', 'WB_Dissolved Oxygen (mg/L)', 'WB_Biochemical Oxygen Demand (mg/L)', 'WB_Turbidity (NTU)', 'WB_Temperature', 'WB_pH_min', 'WB_pH_max', 'WB_Salinity_min', 'WB_Salinity_max', 'WB_Dissolved Oxygen (mg/L)_min', 'WB_Dissolved Oxygen (mg/L)_max', 'WB_Biochemical Oxygen Demand (mg/L)_min', 'WB_Biochemical Oxygen Demand (mg/L)_max', 'WB_Turbidity (NTU)_min', 'WB_Turbidity (NTU)_max', 'WB_Temperature_min', 'WB_Temperature_max']


,Fish ID,Species,Common Name,Lake Name,Kingdom,Phylum,Class,Order,Family,Genus,...,WB_Salinity_min,WB_Salinity_max,WB_Dissolved Oxygen (mg/L)_min,WB_Dissolved Oxygen (mg/L)_max,WB_Biochemical Oxygen Demand (mg/L)_min,WB_Biochemical Oxygen Demand (mg/L)_max,WB_Turbidity (NTU)_min,WB_Turbidity (NTU)_max,WB_Temperature_min,WB_Temperature_max
0,F0001,Copella arnoldi,Splash tetra,Amazon Basin,Animalia,Chordata,Actinopterygii,Characiformes,Lebiasinidae,Copella,...,0.0,0.0,5.5,5.5,2.0,2.0,4.0,4.0,27.0,27.0
1,F0001,Copella arnoldi,Splash tetra,Demerara River Wismar,Animalia,Chordata,Actinopterygii,Characiformes,Lebiasinidae,Copella,...,0.0,0.0,6.0,6.0,3.0,3.0,6.0,6.0,27.0,27.0
2,F0001,Copella arnoldi,Splash tetra,Yarakita River,Animalia,Chordata,Actinopterygii,Characiformes,Lebiasinidae,Copella,...,0.0,0.0,5.0,5.0,2.0,2.0,3.0,3.0,26.5,26.5
3,F0001,Copella arnoldi,Splash tetra,Arauau River,Animalia,Chordata,Actinopterygii,Characiformes,Lebiasinidae,Copella,...,0.0,0.0,5.2,5.2,2.0,2.0,4.0,4.0,27.0,27.0
4,F0001,Copella arnoldi,Splash tetra,"Kurupung River, Upper Mazaruni District",Animalia,Chordata,Actinopterygii,Characiformes,Lebiasinidae,Copella,...,0.0,0.0,4.5,4.5,4.0,4.0,20.0,20.0,26.0,26.0


In [84]:
cols_to_drop = [
    'WB_pH', 'WB_Salinity', 'WB_Dissolved Oxygen (mg/L)',
    'WB_Biochemical Oxygen Demand (mg/L)', 'WB_Turbidity (NTU)', 'WB_Temperature'
]

df = df.drop(columns=cols_to_drop, errors='ignore')

# Save to a new CSV
output_file = "Dataset_v2_cleaned_ranges.csv"
df.to_csv(output_file, index=False, encoding="utf-8")

print(f"✅ Cleaned dataset saved as: {output_file}")
print("Rows in cleaned dataset:", len(df))

✅ Cleaned dataset saved as: Dataset_v2_cleaned_ranges.csv
Rows in cleaned dataset: 2342


In [85]:
df = pd.read_csv("Dataset_v2_cleaned_ranges.csv")

# View first 10 rows
df.head(10)

,Fish ID,Species,Common Name,Lake Name,Kingdom,Phylum,Class,Order,Family,Genus,...,WB_Salinity_min,WB_Salinity_max,WB_Dissolved Oxygen (mg/L)_min,WB_Dissolved Oxygen (mg/L)_max,WB_Biochemical Oxygen Demand (mg/L)_min,WB_Biochemical Oxygen Demand (mg/L)_max,WB_Turbidity (NTU)_min,WB_Turbidity (NTU)_max,WB_Temperature_min,WB_Temperature_max
0,F0001,Copella arnoldi,Splash tetra,Amazon Basin,Animalia,Chordata,Actinopterygii,Characiformes,Lebiasinidae,Copella,...,0.0,0.0,5.50,5.50,2.0,2.0,4.0,4.0,27.0,27.0
1,F0001,Copella arnoldi,Splash tetra,Demerara River Wismar,Animalia,Chordata,Actinopterygii,Characiformes,Lebiasinidae,Copella,...,0.0,0.0,6.00,6.00,3.0,3.0,6.0,6.0,27.0,27.0
2,F0001,Copella arnoldi,Splash tetra,Yarakita River,Animalia,Chordata,Actinopterygii,Characiformes,Lebiasinidae,Copella,...,0.0,0.0,5.00,5.00,2.0,2.0,3.0,3.0,26.5,26.5
3,F0001,Copella arnoldi,Splash tetra,Arauau River,Animalia,Chordata,Actinopterygii,Characiformes,Lebiasinidae,Copella,...,0.0,0.0,5.20,5.20,2.0,2.0,4.0,4.0,27.0,27.0
4,F0001,Copella arnoldi,Splash tetra,"Kurupung River, Upper Mazaruni District",Animalia,Chordata,Actinopterygii,Characiformes,Lebiasinidae,Copella,...,0.0,0.0,4.50,4.50,4.0,4.0,20.0,20.0,26.0,26.0
5,F0001,Copella arnoldi,Splash tetra,Suriname River Kabel Station,Animalia,Chordata,Actinopterygii,Characiformes,Lebiasinidae,Copella,...,0.0,0.0,6.00,6.00,3.0,3.0,15.0,15.0,27.0,27.0
6,F0001,Copella arnoldi,Splash tetra,Dalibane Creek,Animalia,Chordata,Actinopterygii,Characiformes,Lebiasinidae,Copella,...,0.0,0.0,5.00,5.00,2.0,2.0,5.0,5.0,27.0,27.0
7,F0001,Copella arnoldi,Splash tetra,Lane Creek,Animalia,Chordata,Actinopterygii,Characiformes,Lebiasinidae,Copella,...,0.0,0.0,5.00,5.00,2.0,2.0,5.0,5.0,27.0,27.0
8,F0001,Copella arnoldi,Splash tetra,Luci River,Animalia,Chordata,Actinopterygii,Characiformes,Lebiasinidae,Copella,...,0.0,0.0,5.00,5.00,2.0,2.0,4.0,4.0,27.0,27.0
9,F0002,Hyphessobrycon pulchripinnis,Lemon tetra,Rio Tapajós (Santarém waterfront),Animalia,Chordata,Actinopterygii,Characiformes,Characidae,Hyphessobrycon,...,0.0,0.0,0.11,1.69,NaN,NaN,NaN,NaN,28.7,31.1
